In [1]:
from selenium.common.exceptions import TimeoutException
from appium.webdriver.common.appiumby import AppiumBy
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from appium.options.android import UiAutomator2Options
from appium import webdriver

import pandas as pd
from datetime import datetime
import time

In [ ]:
# ===================== 설정 =====================
# 정부24 패키지 / 액티비티 (adb로 확인한 값)
APP_PACKAGE = "kr.go.minwon.m"
APP_ACTIVITY = "kr.go.minwon.m.BrowserActivity"

APPIUM_SERVER_URL = "http://127.0.0.1:4723"
DEVICE_NAME = "Android"

# 🔐 금융인증서 6자리 비밀번호 (※ 실제 값으로 수정해서 사용)
CERT_PW = "000000"

# 반복 횟수
REPEAT_COUNT = 10

# 메인화면 기준 로그인 버튼 좌표 (휴대폰 기종마다 좌표 수정 필요)
LOGIN_BTN_X = 813
LOGIN_BTN_Y = 216

# 전체메뉴 버튼 좌표 (로그인 완료 후) (휴대폰 기종마다 좌표 수정 필요)
MENU_BTN_X = 985
MENU_BTN_Y = 266

In [3]:
# ===================== 공통 유틸 함수 =====================
def tap_by_coordinates(driver, x, y, duration_ms=200):
    """좌표 탭: swipe를 start=end로 주면 탭처럼 동작"""
    try:
        driver.swipe(x, y, x, y, duration_ms)
    except Exception as e:
        print(f"   ❌ 좌표 탭 실패: {e}")
        raise


# ===================== 정부24 전용 동작 함수 =====================
def open_login_section(driver):
    """메인 화면에서 로그인 버튼을 좌표 탭"""
    print("📲 [1단계] 메인화면 로그인 버튼 탭")

    # 메인화면 로딩 확인용 요소(혜택알림 링크)
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (
                    AppiumBy.XPATH,
                    '//android.widget.ImageView[@content-desc="혜택알림 메뉴 바로가기 링크"]',
                )
            )
        )
        print("   ✅ 메인화면 로드 확인 완료")
    except TimeoutException:
        print("   ⚠️ 메인화면 확인용 요소를 찾지 못했습니다. 그래도 로그인 버튼 탭 시도")

    # 로그인 버튼 탭 (👉 스크롤 없음)
    try:
        tap_by_coordinates(driver, LOGIN_BTN_X, LOGIN_BTN_Y)
        print(f"   ✅ ({LOGIN_BTN_X}, {LOGIN_BTN_Y}) 위치 로그인 버튼 탭 완료")
    except Exception as e:
        print(f"   ❌ 로그인 버튼 좌표 탭 실패: {e}")
        raise

    time.sleep(2)


def tap_cert_login(driver):
    """
    로그인 선택 화면에서 '금융인증서' 경로로 진입
    - 스크롤 없이 고정 좌표만 사용
    """
    print("📲 [2단계] '금융인증서' 로그인 선택")

    # 1) '금융인증서' 버튼 탭 (x 780, y 1552) (휴대폰 기종마다 좌표 수정 필요)
    try:
        print("   📍 (780, 1552) 금융인증서 버튼 탭")
        tap_by_coordinates(driver, 780, 1552)
    except Exception as e:
        print(f"   ❌ 금융인증서 버튼 탭 실패: {e}")
        raise

    time.sleep(2)

    # 2) 비밀번호 입력 영역 탭 (x 500, y 1172) (휴대폰 기종마다 좌표 수정 필요)
    try:
        print("   📍 (500, 1172) 비밀번호 입력 영역 탭")
        tap_by_coordinates(driver, 500, 1172)
    except Exception as e:
        print(f"   ❌ 비밀번호 입력 영역 탭 실패: {e}")
        raise

    time.sleep(1)


def enter_cert_pw_and_wait_menu(driver, wait, attempt_idx=None):
    """
    [측정 기준 - 금융인증서]
    - 시작: 금융인증서 비밀번호 6자리 중 '6번째 숫자'를 탭한 직후 시간 측정 시작
    - 끝: 메인화면이 로드되어 전체메뉴(좌표 x=985, y=266)를 클릭할 수 있는 시점
          (혜택알림 메뉴 요소가 보이는 순간을 기준으로 함)
    """
    print("🔐 [3단계] 금융인증서 비밀번호 입력 및 메인화면 대기(측정)")

    start_time = None

    # 1) 자동 입력 시도 (Appium에서 숫자 버튼이 보이는 경우)
    pw = CERT_PW.strip()
    if pw and pw.isdigit() and len(pw) == 6:
        try:
            print("   🤖 금융인증서 비밀번호 자동 입력 시도")
            for idx, digit in enumerate(pw):
                # Appium Inspector에서 본 것처럼 content-desc가 숫자("7")인 버튼 사용
                selector = f'new UiSelector().description("{digit}")'
                btn = WebDriverWait(driver, 15).until(
                    EC.element_to_be_clickable(
                        (AppiumBy.ANDROID_UIAUTOMATOR, selector)
                    )
                )
                btn.click()
                print(f"      ✅ {idx + 1}번째 숫자 클릭")

                # 6번째(마지막) 숫자 클릭 직후 측정 시작
                if idx == len(pw) - 1:
                    start_time = time.time()
                    print("⏱ 측정 시작 (6번째 숫자 클릭 직후)")
        except TimeoutException as e:
            print("   ⚠️ 숫자 버튼을 찾지 못했습니다. 수동 입력으로 전환합니다.")
            start_time = None
        except Exception as e:
            print(f"   ⚠️ 비밀번호 자동 입력 중 오류 발생: {e}")
            start_time = None
    else:
        print("   ⚠️ CERT_PW 값이 비어있거나 6자리 숫자가 아닙니다. 수동 입력으로 전환합니다.")

    # 2) 자동 입력이 불가능할 때: 수동 입력 + 콘솔 Enter 로 시간 측정
    if start_time is None:
        print("   👉 비밀번호를 휴대폰에서 직접 입력해야 합니다.")
        input(
            "      휴대폰에서 6자리 비밀번호 입력을 모두 마치고, "
            "6번째 숫자를 누르는 순간 PC 콘솔에서 Enter 키를 눌러 주세요..."
        )
        start_time = time.time()
        print("⏱ 측정 시작 (사용자 Enter 시점)")

    # 3) 메인화면 로딩 및 전체메뉴 클릭 가능 시점까지 대기
    try:
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located(
                (
                    AppiumBy.XPATH,
                    '//android.widget.ImageView[@content-desc="혜택알림 메뉴 바로가기 링크"]',
                )
            )
        )
        end_time = time.time()
        elapsed = end_time - start_time

        print("🎉 메인화면 로드 확인 (전체메뉴 클릭 가능 상태)")
        print(f"⏱ 측정 시간: {elapsed:.4f} 초")

        # 측정이 끝난 뒤 전체메뉴를 실제로 한 번 열어 둔다.
        try:
            print(f"   📍 ({MENU_BTN_X}, {MENU_BTN_Y}) 전체메뉴 탭 (로그아웃 준비)")
            tap_by_coordinates(driver, MENU_BTN_X, MENU_BTN_Y)
            time.sleep(2)
        except Exception as e:
            print(f"   ⚠️ 전체메뉴 탭 중 오류 (그래도 측정값은 유지): {e}")

        status_msg = "금융인증서 로그인 후 메인화면 로드 완료"
        return elapsed, status_msg

    except TimeoutException:
        end_time = time.time()
        elapsed = end_time - start_time
        print("   ⚠️ 메인화면 확인용 요소를 찾지 못했습니다.(타임아웃)")
        status_msg = "메인화면 미확인(타임아웃)"
        return elapsed, status_msg


def logout_to_main(driver):
    """
    [로그아웃]
    - 전제: 이미 '전체메뉴'가 열린 상태에서 시작
    - 동작: 스크롤 2번 → 로그아웃 버튼 좌표 탭 → 메인화면 복귀 확인
    """
    print("\n🔚 [로그아웃] (전체메뉴 화면 기준) 스크롤 → 로그아웃 → 메인 복귀")

    try:
        # 1) 스크롤 두 번
        print("   ↕ 스크롤 1회: (1040, 1825) → (1040, 242)")
        driver.swipe(1040, 1825, 1040, 242, 500)  # 휴대폰 기종에 따라 좌표 수정 필요
        time.sleep(2)

        print("   ↕ 스크롤 2회: (1040, 1825) → (1040, 242)")
        driver.swipe(1040, 1825, 1040, 242, 500)  # 휴대폰 기종에 따라 좌표 수정 필요
        time.sleep(2)

        # 2) 로그아웃 버튼 좌표 탭
        print("   📍 로그아웃 좌표 탭 (502, 1811)")
        tap_by_coordinates(driver, 502, 1811)  # 휴대폰 기종에 따라 좌표 수정 필요

        # 3) 메인화면 복귀 확인 (혜택알림 요소 기준)
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located(
                    (
                        AppiumBy.XPATH,
                        '//android.widget.ImageView[@content-desc="혜택알림 메뉴 바로가기 링크"]',
                    )
                )
            )
            print("   ✅ 로그아웃 후 메인화면 복귀 확인")
        except TimeoutException:
            print("   ⚠️ 로그아웃 후 메인화면 요소를 찾지 못했습니다. (다음 회차는 진행 시도)")

    except Exception as e:
        print(f"   ❌ 로그아웃 동작 중 오류 발생: {e}")
        raise


# ===================== 로그인 1회 시나리오 =====================
def perform_login_once(driver, wait, attempt_idx=None):
    """
    로그인 시나리오 1회:
    메인 로그인 버튼 → 금융인증서 경로 진입 →
    비밀번호 6번째 숫자 클릭 시점(측정 시작) → 메인화면 로드 및 전체메뉴 클릭 가능(측정 종료)
    """
    print("\n🚀 [로그인 시나리오] 1회 시작")

    open_login_section(driver)
    tap_cert_login(driver)
    elapsed, status_msg = enter_cert_pw_and_wait_menu(driver, wait, attempt_idx)

    return elapsed, status_msg


# ===================== 메인 테스트 + CSV 저장 =====================
def test_login_minwon(repeat_count=REPEAT_COUNT):
    options = UiAutomator2Options()
    options.platform_name = "Android"
    options.device_name = DEVICE_NAME
    options.app_package = APP_PACKAGE
    options.app_activity = APP_ACTIVITY
    options.automation_name = "UiAutomator2"
    options.new_command_timeout = 300
    options.no_reset = True  # 필요에 따라 False 변경

    print("--- [정부24] 금융인증서 로그인 성능 테스트 (반복) ---")
    driver = webdriver.Remote(APPIUM_SERVER_URL, options=options)
    wait = WebDriverWait(driver, 20)

    results = []

    try:
        for i in range(1, repeat_count + 1):
            print("\n" + "=" * 60)
            print(f"🔁 로그인 시도 {i}/{repeat_count}")
            print("=" * 60)

            elapsed, status_msg = perform_login_once(driver, wait, attempt_idx=i)

            results.append(
                {
                    "회차": i,
                    "측정시각": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    "로그인반응속도(초)": round(elapsed, 4),
                    "팝업메시지": status_msg,
                }
            )

            # 마지막 회차가 아니면 로그아웃 후 다음 회차 준비
            if i < repeat_count:
                print("\n📴 [다음 회차 준비] 로그아웃 진행")
                try:
                    logout_to_main(driver)
                except Exception as e:
                    print(f"   ❌ 로그아웃 중 오류 발생 (다음 회차 진행 불가): {e}")
                    break

    finally:
        print("\n🧹 드라이버 종료")
        driver.quit()

    # ===================== CSV 저장 =====================
    if results:
        df = pd.DataFrame(results)

        # 요약 행 추가 (평균, 최소, 최대, 표준편차)
        elapsed_list = [row["로그인반응속도(초)"] for row in results]
        mean_val = sum(elapsed_list) / len(elapsed_list)
        min_val = min(elapsed_list)
        max_val = max(elapsed_list)
        std_val = (sum((x - mean_val) ** 2 for x in elapsed_list) / len(elapsed_list)) ** 0.5

        summary_row = {
            "회차": "요약",
            "측정시각": "-",
            "로그인반응속도(초)": round(mean_val, 4),
            "팝업메시지": (
                f"평균/최소/최대/표준편차 = "
                f"{mean_val:.4f} / {min_val:.4f} / {max_val:.4f} / {std_val:.4f}"
            ),
            "평균(초)": round(mean_val, 4),
            "최소(초)": min_val,
            "최대(초)": max_val,
            "표준편차(초)": std_val,
        }

        df = pd.concat([df, pd.DataFrame([summary_row])], ignore_index=True)


        # 🔻 CSV 저장 직전에 '팝업메시지' 컬럼 제거
        if "팝업메시지" in df.columns:
            df = df.drop(columns=["팝업메시지"])

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_name = f"minwon24_cert_login_perf_{repeat_count}runs_{timestamp}.csv"
        df.to_csv(file_name, index=False, encoding="utf-8-sig")

        print(f"\n✅ CSV 저장 완료! 파일명: {file_name}")
        print(df)
    else:
        print("ℹ️ 저장할 측정 결과가 없어 CSV는 생성되지 않습니다.")

In [4]:
# 함수 실행

if __name__ == "__main__":
    test_login_minwon()

--- [정부24] 금융인증서 로그인 성능 테스트 (반복) ---

🔁 로그인 시도 1/10

🚀 [로그인 시나리오] 1회 시작
📲 [1단계] 메인화면 로그인 버튼 탭
   ✅ 메인화면 로드 확인 완료
   ✅ (813, 216) 위치 로그인 버튼 탭 완료
📲 [2단계] '금융인증서' 로그인 선택
   📍 (780, 1552) 금융인증서 버튼 탭
   📍 (500, 1172) 비밀번호 입력 영역 탭
🔐 [3단계] 금융인증서 비밀번호 입력 및 메인화면 대기(측정)
   🤖 금융인증서 비밀번호 자동 입력 시도
      ✅ 1번째 숫자 클릭
      ✅ 2번째 숫자 클릭
      ✅ 3번째 숫자 클릭
      ✅ 4번째 숫자 클릭
      ✅ 5번째 숫자 클릭
      ✅ 6번째 숫자 클릭
⏱ 측정 시작 (6번째 숫자 클릭 직후)
🎉 메인화면 로드 확인 (전체메뉴 클릭 가능 상태)
⏱ 측정 시간: 1.1810 초
   📍 (985, 266) 전체메뉴 탭 (로그아웃 준비)

📴 [다음 회차 준비] 로그아웃 진행

🔚 [로그아웃] (전체메뉴 화면 기준) 스크롤 → 로그아웃 → 메인 복귀
   ↕ 스크롤 1회: (1040, 1825) → (1040, 242)
   ↕ 스크롤 2회: (1040, 1825) → (1040, 242)
   📍 로그아웃 좌표 탭 (502, 1811)
   ✅ 로그아웃 후 메인화면 복귀 확인

🔁 로그인 시도 2/10

🚀 [로그인 시나리오] 1회 시작
📲 [1단계] 메인화면 로그인 버튼 탭
   ✅ 메인화면 로드 확인 완료
   ✅ (813, 216) 위치 로그인 버튼 탭 완료
📲 [2단계] '금융인증서' 로그인 선택
   📍 (780, 1552) 금융인증서 버튼 탭
   📍 (500, 1172) 비밀번호 입력 영역 탭
🔐 [3단계] 금융인증서 비밀번호 입력 및 메인화면 대기(측정)
   🤖 금융인증서 비밀번호 자동 입력 시도
      ✅ 1번째 숫자 클릭
      ✅ 2번째 숫자 클릭
      ✅ 3번째 숫자 클릭
      